In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from PIL import Image
import glob
import time

### 1. Get training dataset in usable format

- images are in jpg format, need to convert these to PIL Images that will then be converted into tensors
  

In [2]:
### class to load and store data
class image_data:   
    def __init__(self):
        self.label_converter = dict()
        self.convert_label_to_int()
        self.train_data    = [] # train data in tensor format
        self.test_data     = [] # test data in tensor format
        self.validate_data = [] # validation data in tensor format
        self.train_labels =  []
        self.test_labels  =  []
        self.image_files = dict()
        self.load_train_data()
        self.load_test_data()

    ### helper function to convert str letters/del/space into numbers 
    def convert_label_to_int(self):
        alphabet = "A/B/C/D/E/F/G/H/I/J/K/L/M/N/O/P/Q/R/S/T/U/V/W/X/Y/Z/del/nothing/space"
        for iii,label in enumerate(alphabet.split("/")):
            self.label_converter[label] = iii
    ### give this an image filename and it will return the transformed tensor 
    def convert_jpg_to_tensor(self,filepath):
        image = Image.open(filepath) 
          
        # Define a transform to convert PIL  
        # image to a Torch tensor 
        transform = transforms.Compose([ 
            transforms.PILToTensor() 
        ]) 
          
        # transform = transforms.PILToTensor() 
        # Convert the PIL image to Torch tensor 
        return transform(image)
    ### load training data into the instance variable train_data
    def load_train_data(self):
        now = time.time()
        train_path = "datasets/asl_alphabet_train/asl_alphabet_train/"
        train_directories = glob.glob(train_path+"/*")
        n_files = 0
        print(" ----- Loading training dataset -----")
        for dir in train_directories:
            letter = dir.split("/")[-1]
            self.image_files[letter] = []
            
            for image_file in glob.glob(dir+"/*"):
                #print(image_file)
                self.image_files[letter].append(image_file)
                self.train_data.append(self.convert_jpg_to_tensor(image_file))
                self.train_labels.append(self.label_converter[letter])
                n_files +=1
            print("Finished importing %s"%letter)
        #print("Found training file directories - ", train_filepaths)
        print("Done with training dataset - converted %i files. Took %f seconds"%(n_files, time.time()-now))
        return
    ### load test data into the instance variable train_data
    def load_test_data(self):
        now = time.time()
        test_path = "datasets/asl_alphabet_test/asl_alphabet_test/"
        test_directories = glob.glob(test_path+"/*")
        n_files = 0
        print("----- Loading test dataset -----")
        
        for image_file in test_directories:
            letter = image_file.split("_")[-2].split("/")[-1]
            self.image_files[letter] = []
            self.image_files[letter].append(image_file)
            self.test_data.append(self.convert_jpg_to_tensor(image_file))
            self.test_labels.append(self.label_converter[letter])
            n_files +=1
        print("Done with test dataset - converted %i files. Took %f seconds"%(n_files, time.time()-now))
        return

    

In [3]:
id = image_data()


 ----- Loading training dataset -----
Finished importing R
Finished importing U
Finished importing I
Finished importing N
Finished importing G
Finished importing Z
Finished importing T
Finished importing S
Finished importing A
Finished importing F
Finished importing O
Finished importing H
Finished importing del
Finished importing nothing
Finished importing space
Finished importing M
Finished importing J
Finished importing C
Finished importing D
Finished importing V
Finished importing Q
Finished importing X
Finished importing E
Finished importing B
Finished importing K
Finished importing L
Finished importing Y
Finished importing P
Finished importing W
Done with training dataset - converted 87000 files. Took 58.728651 seconds
----- Loading test dataset -----
datasets/asl_alphabet_test/asl_alphabet_test/F_test.jpg
datasets/asl_alphabet_test/asl_alphabet_test/G_test.jpg
datasets/asl_alphabet_test/asl_alphabet_test/L_test.jpg
datasets/asl_alphabet_test/asl_alphabet_test/M_test.jpg
datasets/